# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
!python -m pip install scikit-learn --upgrade
# !python -m pip install nltk

In [47]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support, make_scorer, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [45]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con=engine)
X = df.loc[:, 'message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """
    INPUT:
    OUTPUT:
    """
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Split text into words using NLTK
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    # Lemmatize verbs by specifying pos
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v').strip() for w in lemmed]
    # Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in lemmed]
    tokenized = stemmed
    return tokenized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
random_forest_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
random_forest_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001A513FCA790>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = random_forest_pipeline.predict(X_test)

In [9]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      4983
               request       0.85      0.49      0.62      1123
                 offer       0.00      0.00      0.00        33
           aid_related       0.75      0.69      0.72      2701
          medical_help       0.71      0.07      0.13       539
      medical_products       0.69      0.08      0.14       325
     search_and_rescue       0.89      0.05      0.09       175
              security       0.33      0.01      0.02       111
              military       0.31      0.02      0.04       217
           child_alone       0.00      0.00      0.00         0
                 water       0.87      0.38      0.53       429
                  food       0.82      0.65      0.72       708
               shelter       0.82      0.36      0.50       580
              clothing       0.67      0.11      0.20        87
                 money       0.60      

### 6. Improve your model
Use grid search to find better parameters. 

In [24]:
# USE THIS PARAMETERS AND GRIDSEARCHCV WHEN COMPUTATIONAL POWER AVAILABLE
# parameters = {
#     'vect__ngram_range': ((1, 1), (1, 2)),
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__max_features': (None, 5000, 10000),
#     'tfidf__use_idf': (True, False),
#     'clf__estimator__n_estimators': [50, 100, 200],
#     'clf__estimator__min_samples_split': [2, 3, 4],
# }
# rf_cv_gs = GridSearchCV(random_forest_pipeline, param_grid=parameters, verbose=3, cv=2, return_train_score=True, 
#                         refit='f1_macro', scoring = ['accuracy', 'precision', 'recall', 'f1_macro'])

parameters = {
    'clf__estimator__n_estimators': [4, 5]
}

scoring = {"Accuracy": "accuracy", "Precision": make_scorer(precision_score, average='macro', zero_division=0), 
           "Recall": make_scorer(recall_score, average='macro', zero_division=0), 
           "F1": make_scorer(f1_score, average='macro', zero_division=0)}

rf_cv_gs = GridSearchCV(random_forest_pipeline, param_grid=parameters, verbose=3, cv=2, return_train_score=True, 
                        refit='F1', scoring = scoring)

In [25]:
rf_cv_gs.fit(X_train, y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV 1/2] END clf__estimator__n_estimators=4; Accuracy: (train=0.627, test=0.219) F1: (train=0.782, test=0.199) Precision: (train=0.967, test=0.497) Recall: (train=0.663, test=0.147) total time= 3.4min
[CV 2/2] END clf__estimator__n_estimators=4; Accuracy: (train=0.633, test=0.209) F1: (train=0.779, test=0.197) Precision: (train=0.964, test=0.515) Recall: (train=0.661, test=0.148) total time= 3.5min
[CV 1/2] END clf__estimator__n_estimators=5; Accuracy: (train=0.755, test=0.195) F1: (train=0.881, test=0.270) Precision: (train=0.962, test=0.456) Recall: (train=0.816, test=0.217) total time= 3.5min
[CV 2/2] END clf__estimator__n_estimators=5; Accuracy: (train=0.744, test=0.189) F1: (train=0.879, test=0.256) Precision: (train=0.961, test=0.467) Recall: (train=0.813, test=0.204) total time= 3.5min


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001A513FCA790>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [4, 5]}, refit='F1',
             return_train_score=True,
             scoring={'Accuracy': 'accuracy',
                      'F1': make_scorer(f1_score, average=macro, zero_division=0),
                      'Precision': make_scorer(precision_score, average=macro, zero_division=0),
                      'Recall': make_scorer(recall_score, average=macro, zero_division=0)},
             verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [26]:
best_estimator = rf_cv_gs.best_estimator_
best_estimator

In [27]:
y_pred = best_estimator.predict(X_test)

In [28]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      4983
               request       0.71      0.51      0.59      1123
                 offer       0.00      0.00      0.00        33
           aid_related       0.69      0.65      0.67      2701
          medical_help       0.54      0.19      0.28       539
      medical_products       0.59      0.17      0.26       325
     search_and_rescue       0.37      0.06      0.10       175
              security       0.33      0.03      0.05       111
              military       0.53      0.16      0.24       217
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.52      0.60       429
                  food       0.77      0.61      0.68       708
               shelter       0.73      0.39      0.51       580
              clothing       0.65      0.15      0.24        87
                 money       0.67      

In [44]:
# printing the evaluation scores
rows = []
cvres = rf_cv_gs.cv_results_
for mean_test_Accuracy, mean_test_Precision, mean_test_Recall, mean_test_F1, params in zip(cvres['params'], 
                                                                                           cvres['mean_test_Accuracy'], 
                                                                                           cvres['mean_test_Precision'], 
                                                                                           cvres['mean_test_Recall'], 
                                                                                           cvres['mean_test_F1']):
    rows.append([mean_test_Accuracy, mean_test_Precision, mean_test_Recall, mean_test_F1, params])
scores = pd.DataFrame(rows, columns=["Params", "Accuracy", "Precision", "Recall", "F1"])
scores

,Params,Accuracy,Precision,Recall,F1
0,{'clf__estimator__n_estimators': 4},0.213872,0.506220,0.147520,0.198138
1,{'clf__estimator__n_estimators': 5},0.191947,0.461886,0.210438,0.262890


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [33]:
decision_tree_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

In [34]:
decision_tree_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001A513FCA790>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])

In [35]:
y_pred = decision_tree_pipeline.predict(X_test)

In [36]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.85      0.86      0.86      4983
               request       0.58      0.53      0.55      1123
                 offer       0.03      0.03      0.03        33
           aid_related       0.65      0.65      0.65      2701
          medical_help       0.38      0.32      0.35       539
      medical_products       0.42      0.37      0.40       325
     search_and_rescue       0.23      0.19      0.21       175
              security       0.08      0.06      0.07       111
              military       0.48      0.34      0.40       217
           child_alone       0.00      0.00      0.00         0
                 water       0.64      0.66      0.65       429
                  food       0.71      0.74      0.72       708
               shelter       0.64      0.58      0.61       580
              clothing       0.56      0.46      0.51        87
                 money       0.39      

In [39]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [40]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):

    def textlength(self, text):
        text_len = len(text.strip())
        return text_len

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.textlength)
        return pd.DataFrame(X_tagged)

In [95]:
# With Column Transformer
X = df.loc[:, ['message', 'genre']]
X_train, X_test, y_train, y_test = train_test_split(X, y)
message = 'message'
genre = 'genre'

full_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('col_xfmr', ColumnTransformer([
                ('vect', CountVectorizer(tokenizer=tokenize), message),
                ('tfidf', TfidfTransformer(), message),
                ('cat', OneHotEncoder(), genre)
            ])),

            ('starting_verb', StartingVerbExtractor()),
            ('txt_length', TextLengthExtractor())
        ])),

        ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])

In [ ]:
full_pipeline.fit(X_train, y_train)

In [ ]:
y_pred = full_pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

In [ ]:
parameters = {
    'tfidf__use_idf': (True, False),
    'clf__estimator__min_samples_split': [2, 3]
}

full_pipeline_cv = GridSearchCV(pipeline, param_grid=parameters, scoring = ['accuracy', 'f1_macro'], refit=False, verbose=True)

In [ ]:
full_pipeline_cv.fit(X_train, y_train)

In [ ]:
full_pipeline_cv.best_params_

In [ ]:
best_estimator = full_pipeline_cv.best_estimator_
y_pred = best_estimator.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

In [ ]:
# printing the evaluation scores
cvres = full_pipeline_cv.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

### 9. Export your model as a pickle file

In [ ]:
import joblib
joblib.dump(random_forest_pipeline, "forest_reg.pkl")
joblib.dump(decision_tree_pipeline, "lin_reg.pkl")
joblib.dump(best_estimator, "tree_reg.pkl")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.